In [1]:
import csv
from functools import reduce
import math
import json

In [2]:
class User: # пользователь
    name = ''
    movie_rates = {}
    avg_rate = 0
    sim_users = {}

In [3]:
def parse_movie_names(row):
    row = list(map(lambda name: name.strip(), row))
    return list(filter(lambda name: name != '', row))

In [4]:
def load_users(filename):
    data_file = open(filename)
    reader = csv.reader(data_file)
    
    movie_names = parse_movie_names(next(reader))
    users = {}
    
    for row in reader:
        user = User()
        user.name = row[0]
        user.movie_rates = dict(zip(movie_names, [int(rate) for rate in row[1:]]))
        
        users[user.name] = user
        
    return users

In [52]:
def calculate_avg_rate(user): # считаем среднее
    valid_rates = list(filter(lambda rate: rate != -1, user.movie_rates.values()))
    return reduce(lambda summ, rate: summ + rate, valid_rates, 0) / len(valid_rates)

In [6]:
def set_metric(users): # составляем метрику сходства
    for first_user in users.values(): 
        first_user.sim_users = {first_user: 0}
        for second_user in users.values():
            if first_user.name == second_user.name:
                continue
            s = 0
            sq1 = 0
            sq2 = 0
            for movie_name in first_user.movie_rates.keys():
                if first_user.movie_rates[movie_name] != -1 and second_user.movie_rates[movie_name] != -1:
                    s += first_user.movie_rates[movie_name] * second_user.movie_rates[movie_name]
                    sq1 += math.pow(first_user.movie_rates[movie_name], 2)
                    sq2 += math.pow(second_user.movie_rates[movie_name], 2)
            sq1 = math.sqrt(sq1)
            sq2 = math.sqrt(sq2)
            sq = sq1 * sq2
            first_user.sim_users[second_user] = s / (sq) if sq != 0 else 0 

In [31]:
def compute_missed_rates(user, k):
    sorted_sim_users = s = [u[0] for u in sorted(user.sim_users.items(), key=lambda d: d[1], reverse=True)]
    
    for (movie_name, movie_rate) in user.movie_rates.items():
        if (movie_rate != -1):
            continue
        
        summ1 = 0
        summ2 = 0
        for other_user in sorted_sim_users[:k]:
            if(other_user.movie_rates[movie_name] != -1):
                summ1 += user.sim_users[other_user] * (other_user.movie_rates[movie_name] - other_user.avg_rate)
                summ2 += user.sim_users[other_user]
                
        user.movie_rates[movie_name] = round(user.avg_rate + summ1/summ2, 3)

In [51]:
users = load_users('./data.csv')
set_metric(users)
for user in users.values():
    user.avg_rate = calculate_avg_rate(user)
user_nm = 'User 1'
required_user = users[user_nm]
print('---------')
compute_missed_rates(required_user, 7)
print(required_user.movie_rates['Movie 1'])
'''
for movie_name, movie_rate in required_user.movie_rates.items():
    print(movie_name, movie_rate)
'''

User 1 62 21 2.9523809523809526
User 2 79 24 3.2916666666666665
User 3 70 25 2.8
User 4 66 21 3.142857142857143
User 5 86 26 3.3076923076923075
User 6 79 25 3.16
User 7 72 25 2.88
User 8 79 24 3.2916666666666665
User 9 73 26 2.8076923076923075
User 10 68 25 2.72
User 11 75 24 3.125
User 12 100 26 3.8461538461538463
User 13 82 26 3.1538461538461537
User 14 75 25 3.0
User 15 81 24 3.375
User 16 92 27 3.4074074074074074
User 17 66 22 3.0
User 18 73 27 2.7037037037037037
User 19 75 24 3.125
User 20 70 22 3.1818181818181817
User 21 74 25 2.96
User 22 74 23 3.217391304347826
User 23 78 23 3.391304347826087
User 24 76 27 2.814814814814815
User 25 69 25 2.76
User 26 73 25 2.92
User 27 80 24 3.3333333333333335
User 28 76 26 2.923076923076923
User 29 80 25 3.2
User 30 83 28 2.9642857142857144
User 31 83 27 3.074074074074074
User 32 81 27 3.0
User 33 72 26 2.769230769230769
User 34 73 24 3.0416666666666665
User 35 73 23 3.1739130434782608
User 36 75 24 3.125
User 37 78 24 3.25
User 38 72 24 3.0
U

'\nfor movie_name, movie_rate in required_user.movie_rates.items():\n    print(movie_name, movie_rate)\n'